<b>Steps:</b>

1. Use the New York Times API to fetch the new articles based on the keywords
2. Iterate and save all the urls
3. Hit the urls seperately and fetch the html content
4. Use the BeautifulSoap to parse the html
4. Get the content of the url and save the whole content in a single file

https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [ ]:
import json
import sys
import requests
from bs4 import BeautifulSoup
import re
import os
import time

In [ ]:
!{sys.executable} -m pip install beautifulsoup4

In [ ]:
def remove_tag(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [ ]:
def get_data(keyword, folder_name, file_count):
    
    web_url = []
    for i in range(0,100): 
        params = {
             'api-key': '', #ToDo: Add your api-key
             'begin_date': '', #ToDo: Add start date
             'end_date': '', #ToDo: Add end date
             'q': keyword,
             'page' : str(i),
             'fl': "web_url",
        }
        # Sleep for 1 second so that you dont exceed the NY api hit limit per second
        time.sleep(1)
        nytimes_response = requests.get(url='https://api.nytimes.com/svc/search/v2/articlesearch.json', params= params)
        str_nytimes_response = nytimes_response.content.decode("utf-8") 
        json_nytimes_response = json.loads(str_nytimes_response)
        print("iteration: "+ str(i))
        
        # iterate the respone and if docs is present inside response then append the web url
        if(json_nytimes_response['response']['docs'] != []):
            for i in json_nytimes_response['response']['docs']:
                web_url.append(i['web_url'])
            print(len(web_url))
        else :
            break
    # For each url obtained hit the url and get the content         
    for url in web_url:
        url_response = requests.get(url)
        # Use BeautifulSoup to parse the html
        soup = BeautifulSoup(url_response.content, 'html.parser')
        
        # Get the story content from the paragraph. Note this depends on the websites and for new york times it was mostly found to be story-content
        story = soup.findAll("p", class_="story-content")
        e2kc3sl0 = soup.findAll("p", class_="e2kc3sl0")

        web_text = ''

        # Append the various paragraphs and make it a complete content
        for text in story:
            temp = remove_tag(str(text))
            web_text = web_text+" "+temp
            
        for e in e2kc3sl0:
            t = remove_tag(str(e))
            web_text = web_text+" "+t
        
        file_count = file_count + 1;
        #Save the file in the notation [end_data]_[file_count].txt
        file_name = keyword+params['end_date']+"_"+str(file_count)+".txt"
        
        print("File Name: "+file_name)
        print("url: "+url)
        
        filepath = os.path.join(os.getcwd()+'/'+folder_name, file_name)
        if not os.path.exists(os.getcwd()+'/'+folder_name):
            os.makedirs(os.getcwd()+'/'+folder_name)
        f = open(filepath, "w")
        f.write(web_text)
        f.close()


In [ ]:
keywords = ["blockchain","ethereum","r3 corda","bitcoin","hyperledger","crypto", "initial coin offering","ether","dapp","litecoin"]
for key in keywords:
    get_data(key,"",0) # toDo: Enter folder Name